In [ ]:
import ee
import geemap

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
import geemap.colormaps as cm

import seaborn as sns
import geopandas as gpd

import datetime

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

# Functions

In [ ]:
# Mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask) \
        .divide(10000) \
        .copyProperties(image, ['system:time_start']) 

# Mask out water
def maskWater(image):
    return image.updateMask(waterMask.select('water_mask').lt(1))

# Function to filter images by NDSI_Snow_Cover value < 5
def maskS2snow(image):

    mask = image.select('MSK_SNWPRB').lt(0.009) \
    
    return image.updateMask(mask) \
            .copyProperties(image, ['system:time_start'])

# Mask white to remove cloudy or snowy pixels
def maskWhite(image):
    grayscale = image.expression(
            '(.3 * 1e4 * R) + (.59 * 1e4 * G) + (.11 * 1e4 * B)', {
            # '(R + G + B) / 3', {
            'R': image.select('B4'),
            'G': image.select('B3'),
            'B': image.select('B2')
        })
    
    white_mask = grayscale.lte(2000)

    return image.updateMask(white_mask).copyProperties(image,['system:time_start'])

def clp(image):
    return image.clip(aoi)

# Make an NDVI band
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi).copyProperties(image, ['system:time_start'])

# Get yearly statistics for the chosen index
def annual_images(y):
    range_year = ee.Filter.calendarRange(y, y, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(lambda image: image.addBands(image.metadata('system:time_start').divide(3.154e10)))) # Needed for linear regression 
    
    # Print out the number of images in the ImageCollection for each year
    num_images = filtered_dataset.size()
    
    
    # Choose the reducer based on the analysis choice
    if analysis == 'mean':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        )
    elif analysis == 'min' or analysis == 'max':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.minMax(),
            sharedInputs=True
        )
    elif analysis == 'median':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.median(),
            sharedInputs=True
        )

    # Use the combined reducer to get the statistics
    stats = filtered_dataset.reduce(reducer)
    return stats.set('year', y).set('num', num_images)

# get the seasonal (one year) trend
def intrayear(index_collection):
    range_year = ee.Filter.calendarRange(start_year, end_year, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(lambda image: image.addBands(image.metadata('system:time_start').divide(3.154e10)))) # Needed for linear regression 
    
    num_images = filtered_dataset.size()

    # return filtered_dataset
    return filtered_dataset.set('year', start_year).set('num', num_images)

# function to get the trend of the trend
def annual_trend(y):
    range_year = ee.Filter.calendarRange(y, y, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(lambda image: image.addBands(image.metadata('system:time_start').divide(3.154e10)))) # Needed for linear regression 
    
    # Print out the number of images in the ImageCollection for each year
    num_images = filtered_dataset.size()
    
    # Use the combined reducer to get the statistics
    stats = filtered_dataset.reduce(reducer = ee.Reducer.linearFit())
    return stats.set('year', y).set('num', num_images)

def createTimeBand(image):   
    return image.addBands(image.metadata('system:time_start').divide(3.154e10))

# get the mean NDVI for a region
def meanNDVI(image): 
    ndvi = image.select('NDVI')
    ndvi_mean = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10
    ).get('NDVI')

    ndvi_mean = ee.Number(ndvi_mean)
    
    return image.set('ndvi_mean', ndvi_mean)

# Build collection

Define AOI and build image collection

In [ ]:
HYBAS_ID = 8100362560

aoi = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_10").filter(ee.Filter.eq('HYBAS_ID', HYBAS_ID))

In [ ]:
# Get water mask
waterMask = (
    ee.ImageCollection('MODIS/006/MOD44W') 
    .filter(ee.Filter.date('2015-01-01', '2015-01-02')) 
    .select('water_mask') \
    .first()
)
# Get Sentinel 2 harmonized images
dataset = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                #  Filter by year
                  .filter(ee.Filter.calendarRange(2019,2023,'year'))
                #  Filter by month
                  .filter(ee.Filter.calendarRange(6,9,'month'))
                #  Pre-filter to get less cloudy granules.
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                  .filterBounds(aoi)
                #   // This one's Toolik
                # .filterBounds(ee.Geometry.Point(-149.5427, 68.6267).buffer(500))
                #   // This one's Russian tree tracks
                # .filterBounds(ee.Geometry.Point(133.16008, 66.82386).buffer(1000))
                  .map(clp)
                  .map(maskS2clouds)
                  .map(maskS2snow)
                  .map(maskWhite)
                  .map(maskWater)
                  .map(addNDVI)
)

In [ ]:
long = aoi.geometry().centroid().coordinates().get(0).getInfo()
lat = aoi.geometry().centroid().coordinates().get(1).getInfo()

Map = geemap.Map(center = (lat, long), zoom = 11)

### Do analysis
select your index and time frame

In [ ]:
# Pick your index
index = 'NDVI'
# Choose 'mean', 'median', 'min', or 'max' for analysis
analysis = 'max'  

# limit beyond your ImageCollection
start_year = 2019
end_year = 2023
start_month = 6
end_month = 8

# image collection with index band
index_collection = dataset.select(index)  

# Generate list of years
years = ee.List.sequence(start_year, end_year)

#### Skip to different sections to:
- view spectral trends on the map <br>
- plot the index values of different regions <br>
- perform a KMeans cluster

# View Trend

Temporal trend: change in index over time

In [ ]:
if start_year == end_year:
    intrayear_collection = intrayear(index_collection)

    item = intrayear_collection.getInfo()
    print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

    # Get linear fit to pixelwise trend of annual max NDVI
    trend = intrayear_collection.select(['system:time_start',
                                index
                                ]).reduce(ee.Reducer.linearFit())

else:

    # Map over years to get yearly statistics
    yearwise_ndvi = years.map(annual_images)

    for item in yearwise_ndvi.getInfo():
        print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

        yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

        # Get linear fit to pixelwise trend of annual max NDVI
        trend = yearCompCol.select(['system:time_start_mean',
                                    f'{index}_{analysis}'
                                    ]).reduce(ee.Reducer.linearFit())



Trend of the trend: change in seasonal trend over time (skip the above cell)

In [ ]:
# Map over years to get annual trend
yearwise_trend = years.map(annual_trend)

for item in yearwise_trend.getInfo():
    print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

yearCompCol = ee.ImageCollection.fromImages(yearwise_trend)

trend = yearCompCol.reduce(ee.Reducer.linearFit())

### Map

To determine the min and max values of the trend, sample and plot the data.

In [ ]:
slope = trend.select('scale')
samples = slope.sample(region=aoi, scale=10, numPixels=500, geometries=True).getInfo()

slope_values = [feature['properties']['scale'] for feature in samples['features']]
coordinates = [feature['geometry']['coordinates'] for feature in samples['features']]

coords = np.array(coordinates)
lons = coords[:, 0]
lats = coords[:, 1]

# Plot the slope values
plt.figure(figsize=(10, 6))
plt.scatter(lons, lats, c=slope_values, cmap='viridis', marker='o')
plt.colorbar(label='Slope')
plt.title('Slope Values Over Region')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# set trend min and max values
tMin = -0.125
tMax = 0.05

image_limit = 5

# view RGB
for image_id in intrayear(dataset).aggregate_array("system:index").getInfo()[0:image_limit]:
        image = intrayear(dataset).filterMetadata("system:index", "equals", image_id).first()
        
        image_RGB = image.select('B4', 'B3', 'B2') 
        RGB_vis_params = {'min': 0.0, 'max': 0.3}
        Map.addLayer(image_RGB, RGB_vis_params, ee.Image(image).date().format('yyyy-MM-dd').getInfo(), True)

# view NDVI
# for image_id in intrayear_collection.aggregate_array("system:index").getInfo()[0:image_limit]:
#         image = intrayear_collection.filterMetadata("system:index", "equals", image_id).first()
#         date_string = ee.Image(image).date().format('yyyy-MM-dd').getInfo()

#         Map.addLayer(image.select('NDVI'), {}, f'{date_string}_NDVI', False)

# view trend
Map.addLayer(trend.select('scale'),
              {'min': tMin, 'max': tMax,
            'palette': ['red', 'white', 'blue']
            },
 'trend')

# add colorbar for trend
Map.add_colorbar_branca(colors=['red', 'white', 'blue'], vmin=tMin, vmax=tMax, layer_name='trend')

Map

# Compare trends within drawn polygons


Run the following cell and draw a region of interest on the map

In [ ]:
# pick the image with the most coverage
image_RGB = dataset.sort('NODATA_PIXEL_PERCENTAGE', False).first().select('B4', 'B3', 'B2')

RGB_vis_params = {'min': 0.0, 'max': 0.3}
Map.addLayer(image_RGB, RGB_vis_params, ee.Image(image_RGB).date().format('yyyy-MM-dd').getInfo(), True)

Map

Calculate mean NDVI of the area for each date and convert to a data frame.

In [ ]:
# clip to polygon
aoi = ee.FeatureCollection(Map.draw_features)
index_collection = index_collection.map(clp)

# average NDVI
ndvi_collection = index_collection.map(meanNDVI)

# Map.addLayer(index_collection.first(), {}, "clip")
# Map

In [ ]:
image_list = ndvi_collection.getInfo()
data = []

# add date and mean NDVI to data frame
for img in image_list['features']:
    properties = img['properties']
    data.append({
            'date': ee.Date(properties['system:time_start']).format('YYYY-MM-dd').getInfo(),
            'ndvi_mean': properties.get('ndvi_mean')
        })
    
df = pd.DataFrame(data, columns=['date', 'ndvi_mean'])

In [ ]:
# save csv 
df.to_csv('/sciclone/home/aekastning/pycogss_recipes/spectral-change-detector/output/8100362560_wt.csv')

Once you have drawn your AOIs and saved their NDVI values, you can load two CSVs to compare their data.

In [ ]:
# load NDVI values for water track
wt = pd.read_csv('/sciclone/home/aekastning/pycogss_recipes/spectral-change-detector/output/8100362730_wt.csv')

# load NDVI values for intertrack region
intertrack = pd.read_csv('/sciclone/home/aekastning/pycogss_recipes/spectral-change-detector/output/8100362730_slp.csv')

In [ ]:
# convert date to datetime
wt['date'] = pd.to_datetime(wt['date'])
intertrack['date'] = pd.to_datetime(intertrack['date'])

# add ordinal date column to assist with plotting
wt['date_ordinal'] = wt['date'].apply(lambda x: x.toordinal())
intertrack['date_ordinal'] = intertrack['date'].apply(lambda x: x.toordinal())

In [ ]:
wt['region'] = 'track'
intertrack['region'] = 'intertrack'

In [ ]:
# linear regression
sns.set_theme(style = 'white')
sns.lmplot(x='date_ordinal', y='ndvi_mean', data=intertrack)

plt.xlabel('Date')
plt.ylabel('Mean NDVI')
plt.title('Temporal change in NDVI of inter-track region')

interval = 21
tick_positions = intertrack['date_ordinal'][::interval]
tick_labels = intertrack['date'][::interval].dt.strftime('%Y-%m-%d')

plt.xticks(ticks=tick_positions, labels=tick_labels,)

plt.gca().xaxis.grid(False)

plt.show()

### plot inverse vs normal greening
run above cells to extract NDVI values for water track and intertrack regions.
then, save data as either inverse or normal greening

In [ ]:
inverse = pd.concat([wt, intertrack], ignore_index=True)

In [ ]:
normal = pd.concat([wt, intertrack], ignore_index=True)

linear regression of NDVI over time for EITHER inverse or normal

In [ ]:
sns.set_theme(style = 'white')
fig_inv = sns.lmplot(x='date_ordinal', y='ndvi_mean', data=inverse, hue = 'region')

plt.xlabel('Date')
plt.ylabel('Mean NDVI')
plt.title('Temporal NDVI Trends of Inverse Greening Water Tracks and Surrounding Hillslope')

interval = 85
tick_positions = inverse['date_ordinal'][::interval]
tick_labels = inverse['date'][::interval].dt.strftime('%Y-%m-%d')

plt.xticks(ticks=tick_positions, labels=tick_labels,)

plt.gca().xaxis.grid(False)

plt.show()

In [ ]:
# save figure (can't do lmplot with subplot axes)
fig_inv.savefig('/sciclone/home/aekastning/pycogss_recipes/spectral-change-detector/output/inv.png')

repeat the above steps for other region

In [ ]:
# filter dfs to for June ndvi
normalJune = normal[normal['date'].dt.month == 6]
inverseJune = inverse[inverse['date'].dt.month == 6]

In [ ]:
# big fig

sns.set_theme(style='white')

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Normal Greening linear regression
normal_img = plt.imread('/sciclone/home/aekastning/pycogss_recipes/spectral-change-detector/output/normal.png')
axes[0, 0].imshow(normal_img)
axes[0, 0].axis('off') 

# inverse greening linear regression
inverse_img = plt.imread('/sciclone/home/aekastning/pycogss_recipes/spectral-change-detector/output/inv.png')
axes[0, 1].imshow(inverse_img)
axes[0, 1].axis('off')

# June Greeness
sns.boxplot(x='region', y='ndvi_mean', data=normalJune, ax=axes[1, 0])
axes[1, 0].set_xlabel('Region')
axes[1, 0].set_ylabel('June NDVI')
axes[1, 0].set_title('June Greenness by Region')

sns.boxplot(x='region', y='ndvi_mean', data=inverseJune, ax=axes[1, 1])
axes[1, 1].set_xlabel('Region')
axes[1, 1].set_ylabel('June NDVI')
axes[1, 1].set_title('June Greenness by Region with Inverse Greening')

plt.tight_layout()
plt.show()

# K Means Clustering


Make an image with June Greeness, Greening Season Trend, Interannual greening trend, and Trend of Trend as bands

In [ ]:
# pick the image with the most coverage

trend_image = dataset.sort('NODATA_PIXEL_PERCENTAGE', False).first().select('B4', 'B3', 'B2')

In [ ]:
# trend of trend
yearwise_trend = years.map(annual_trend)

for item in yearwise_trend.getInfo():
    print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

yearCompCol = ee.ImageCollection.fromImages(yearwise_trend)

trend = yearCompCol.reduce(ee.Reducer.linearFit())

scale = trend.select('scale')

trend_image = trend_image.addBands(scale.rename('trend'))

In [ ]:
# calculate mean growing season trend 2019-2023

trend = yearCompCol.reduce(ee.Reducer.mean())

scale = trend.select('scale_mean')

trend_image = trend_image.addBands(scale.rename('season'))

In [ ]:
# calculate change in annual max ndvi over time

yearwise_ndvi = years.map(annual_images)

for item in yearwise_ndvi.getInfo():
    print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

trend = yearCompCol.select(['system:time_start_mean',
                            'NDVI_max'
                             ]).reduce(ee.Reducer.linearFit())

scale = trend.select('scale')
    
trend_image = trend_image.addBands(scale.rename('temporal'))

In [ ]:
# calculate June Greenness 2019-2023

end_month = 6

yearwise_ndvi = years.map(annual_images)

for item in yearwise_ndvi.getInfo():
    print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

# Get linear fit to pixelwise trend of annual max NDVI
trend = yearCompCol.select(['system:time_start_mean',
                                    f'{index}_{analysis}'
                                    ]).reduce(ee.Reducer.linearFit())
    
scale = trend.select('scale')
    
trend_image = trend_image.addBands(scale.rename('june'))

training

In [ ]:
img = trend_image.select('temporal', 'season', 'june')

training = img.sample(
    region=aoi,
    scale=10,
    numPixels=5000
)

kmeans

In [ ]:
kmeans = ee.Clusterer.wekaKMeans(4).train(training) # integer = number of clusters

kmeansresult = img.cluster(kmeans)

In [ ]:
# plot RGB
Map.addLayer(trend_image.select('B4', 'B3', 'B2'), {'min': 0.0, 'max': 0.3}, ee.Image(trend_image).date().format('yyyy-MM-dd').getInfo(), True)

# plot clusters
Map.addLayer(kmeansresult.select('cluster'),
              {'min':0, 'max':4,
               # 'palette': cm.palettes.viridis
            },
 'kmeans')

Map

plots

In [ ]:
# convert to df

sample = kmeansresult.addBands(trend_image.select('trend', 'temporal','season','june')).sample(
    region=aoi,
    scale=10,
    numPixels=5000
).getInfo()

data = []

for feature in sample['features']:
    properties = feature['properties']
    data.append([properties['trend'], properties['temporal'], properties['season'], properties['june'], properties['cluster']])
df = pd.DataFrame(data, columns=['trend', 'temporal', 'season', 'june', 'cluster'])


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(df['temporal'], df['season'], df['june'], c=df['cluster'], cmap='gray')

# Labels
ax.set_xlabel('temporal')
ax.set_ylabel('season')
ax.set_zlabel('june')

# Legend
legend1 = ax.legend(*scatter.legend_elements(), title="clusters")
ax.add_artist(legend1)

# Show plot
plt.show()

In [ ]:
fig = plt.figure()

scatter = plt.scatter(df['june'], df['trend'], c=df['cluster'], cmap='gray', edgecolors='k')

# Labels
plt.xlabel('june')
plt.ylabel('trend')

# Legend
legend1 = plt.legend(*scatter.legend_elements(), title="Clusters")

# Show plot
plt.show()

In [ ]:
sns.catplot(kind='box', data=df, y='temporal', x='cluster', palette="gray")
plt.tight_layout()
# plt.title('North Slope (8100362730)')
plt.show()